In [1]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import numpy as np
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import img_to_array
import joblib
from flask import Flask, request, jsonify, render_template
from werkzeug.utils import secure_filename


In [2]:

disease_descriptions = {
    'BA- cellulitis': 'Cellulitis is a painful bacterial infection of the deeper layers of skin.\
         It can start suddenly and may become life threatening without prompt treatment.\nCellulitis symptoms include:\
        \n-pain and tenderness in the affected area.\n-redness or inflammation of your skin.\n-a skin sore or rash that grows quickly.\n-tight, glossy, swollen skin.\n-a feeling of warmth in\
        the affected area.\n-an abscess with pus.\n-fever.\nCellulitis treatment typically involves taking\
        *antibiotics by mouth for at least 5 daysTrusted Source.',

    'BA-impetigo': 'Impetigo starts with red sores or blisters, but the redness may be harder to see on brown and black skin.\
        The sores or blisters quickly burst and often leave crusty, golden-brown patches. \nThe treatment could be:\
        \n*Antibiotics are the first line of treatment for impetigo. The type of antibiotic you get depends on how widespread or severe your lesions are.',

    'FU-athlete-foot': 'Athlete\'s foot (tinea pedis) is a fungal skin infection that usually begins between the toes.\
        It commonly occurs in people whose feet have become very sweaty while confined within tight-fitting shoes.\
        \nSymptoms of athlete’s foot include:\n-a scaly, red rash that usually develops between the toes before spreading.\n-itchiness, which is often most severe after taking off your shoes and socks.\
        \n-blisters in severe cases.\nMost cases of athlete’s foot can be treated with OTC powders, creams, or sprays.\
        There are many options available, including:\n*miconazole (Desenex).\n*tolnaftate (Tinactin).\n*clotrimazole (Lotrimin AF).\n*butenafine (Lotrimin Ultra).\
        \n*terbinafine (Lamisil AT).',

    'FU-nail-fungus': 'Nail fungus is a common infection of the nail. It begins as a white or yellow-brown spot under the tip of your fingernail or toenail.\
        As the fungal infection goes deeper, the nail may discolor, thicken and crumble at the edge.\
        \nMedical treatment options for toenail fungus include:\n*antifungal tablets.\n*antifungal nail cream.\n\
        *nail-softening cream.\n*laser treatment.\n*medical removal of the nail.\
        \nSeveral home remedies may be effective for nail fungus, including:\n*topical ointments containing menthol, such as Vicks VapoRub.\n\
        *tea tree oil.\n*apple cider vinegar.\n*essential oils.',

    'FU-ringworm': 'Signs symptoms may include:\n-itchiness.\n-itchy or scaly patches that are red, brown, or gray, or raised areas of skin called plaques.\n\
        -a round, flat patch of itchy skin.\n-patches that develop blisters or pustules.\n\
        -patches that resemble a ring with deeper color on the outside.\n-patches with edges that are defined and raised.\n\
        -overlapping rings.\n-hair loss.\nTreatment could be:\n*clotrimazole (Lotrimin).\n*miconazole.\n*terbinafine.',

    'PA-cutaneous-larva-migrans': 'Cutaneous larva migrans (CLM) is a serpiginous eruption that can occur anywhere on exposed body parts but is usually confined to the skin of the feet.\
        It is most often caused by dog and cat hookworms, which are types of nematodes (roundworms).\
        \nSymptoms:\n-Red, twisting lesions that grow.\n-Itchiness and discomfort.\
        \n-Swelling.\n-Lesions on the feet and backside.',

    'VI-chickenpox': 'Chickenpox is an infection that causes an itchy, blister-like skin rash.\
        A virus called varicella-zoster causes it. Chickenpox is highly contagious.\
        But it\'s much less common today because there\'s a vaccine that protects you from it.\
        \nChickenpox most commonly occurs in children and usually causes mild symptoms like:\n-blistering rash.\
        \n-fever.\n-headache.\n-red or pink bumps across your body.',

    'VI-shingles': 'Shingles is a painful, usually itchy, rash that develops on one side of the face or body. \
        The rash consists of blisters that typically scab over in 7 to 10 days, clearing up within 2 to 4 weeks.\
        Long-term nerve pain is the most common complication of shingles.\nSymptoms like:\n-fever.\n-headache.\n-chills.\n-muscle weakness.\n-an itchy rash typically on one side of your body.\
        \nAntiviral medications for shingles include:\n*acyclovir.\n*famciclovir.\n*valacyclovir.'  
}
app = Flask(__name__)

# Set paths for dataset and uploads
train_root = 'skin_disease_dataset/train_set'
test_root = 'skin_disease_dataset/test_set'
UPLOAD_FOLDER = 'static/uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # Limit to 16 MB

# Allowed extensions for uploaded files
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif'}

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

# Load the label encoder and other necessary components...
label_encoder = LabelEncoder()

def load_data(root):
    path = []
    labels = []
    for folder_name in os.listdir(root):
        folder_path = os.path.join(root, folder_name)
        if os.path.isdir(folder_path):
            for file_name in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file_name)
                path.append(file_path)
                labels.append(folder_name)
    return pd.DataFrame({'disease path': path, 'disease name': labels})

# Load train and test datasets
train_df = load_data(train_root)
test_df = load_data(test_root)

# Encode labels
label_encoder.fit(train_df['disease name'])
train_df['disease name'] = label_encoder.transform(train_df['disease name'])
test_df['disease name'] = label_encoder.transform(test_df['disease name'])

# Prepare features and labels
X_train_paths = train_df['disease path']
y_train_labels = train_df['disease name']
X_test_paths = test_df['disease path']
y_test_labels = test_df['disease name']

def load_and_preprocess_image(filepath, size=(128, 128)):
    """Load and preprocess an image."""
    try:
        img = Image.open(filepath).convert('RGB')
        img = img.resize(size)
        img_array = img_to_array(img) / 255.0  
        return img_array
    except Exception as e:
        print(f"Error loading image {filepath}: {e}")
        return None

# Load pretrained CNN model for feature extraction
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
feature_extractor_model = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)

def extract_features(image_paths):
    """Extract features from images using the pre-trained model."""
    features_list = []
    for path in image_paths:
        img_array = load_and_preprocess_image(path)
        if img_array is not None:
            img_array_expanded = np.expand_dims(img_array, axis=0)
            feature_vector = feature_extractor_model.predict(img_array_expanded)
            features_list.append(feature_vector.flatten())
    return np.array(features_list)

# Check if the feature files exist and load or extract them accordingly.
if os.path.exists('train_features.npy') and os.path.exists('test_features.npy'):
    X_train_features_loaded = np.load('train_features.npy')
    X_test_features_loaded = np.load('test_features.npy')
else:
    X_train_features_loaded = extract_features(X_train_paths)
    X_test_features_loaded = extract_features(X_test_paths)
    
    np.save('train_features.npy', X_train_features_loaded)
    np.save('test_features.npy', X_test_features_loaded)

# Check if the model file exists and load or train it accordingly.
if os.path.exists('skin_disease_model.pkl'):
    clf_model_loaded = joblib.load('skin_disease_model.pkl')
else:
    param_grid_svm = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': ['scale']}
    
    grid_search_svm_model = GridSearchCV(svm.SVC(class_weight='balanced'), param_grid_svm, scoring='accuracy', cv=5)
    
    grid_search_svm_model.fit(X_train_features_loaded, y_train_labels)
    
    clf_model_loaded_best_estimator_ = grid_search_svm_model.best_estimator_
    
    joblib.dump(clf_model_loaded_best_estimator_, 'skin_disease_model.pkl')

@app.route('/')
def index():
   return render_template('upload_capture.html')

@app.route('/predict', methods=['POST'])
def predict():
    if 'image' not in request.files:
        return jsonify({'error': 'No file part in the request'}), 400

    file = request.files['image']

    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if not allowed_file(file.filename):
        return jsonify({'error': 'File type not allowed'}), 400

    try:
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], secure_filename(file.filename))
        file.save(filepath)

        # Load and preprocess the uploaded image for prediction
        uploaded_image_array = load_and_preprocess_image(filepath)

        if uploaded_image_array is not None:
            uploaded_image_expanded = np.expand_dims(uploaded_image_array, axis=0)  # Add batch dimension
            features_vector = feature_extractor_model.predict(uploaded_image_expanded).flatten().reshape(1, -1)

            # Make prediction using the SVM model
            predicted_label_index = clf_model_loaded.predict(features_vector)[0]

            # Decode the prediction back to the original label
            predicted_disease_name = label_encoder.inverse_transform([predicted_label_index])[0]

            # Get the description for the predicted disease
            disease_description = disease_descriptions.get(predicted_disease_name, 'Description not available.')

            # Return both the predicted disease and the description
            return jsonify({
                'predicted_disease': predicted_disease_name,
                'description': disease_description
            }), 200

        else:
            return jsonify({'error': 'Error processing the image.'}), 500

    except Exception as e:
        print(f"Error saving file: {e}")
        return jsonify({'error': 'Error saving file.'}), 500

if __name__ == '__main__':
   app.run(host="0.0.0.0", port=8000) 

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://192.168.1.114:8000
Press CTRL+C to quit
127.0.0.1 - - [17/Jan/2025 19:16:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2025 19:16:59] "GET /static/js/script.js HTTP/1.1" 304 -
127.0.0.1 - - [17/Jan/2025 19:16:59] "GET /static/css/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [17/Jan/2025 19:17:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2025 19:17:00] "GET /static/images/image.jpg HTTP/1.1" 304 -
127.0.0.1 - - [17/Jan/2025 19:17:01] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [17/Jan/2025 19:17:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2025 19:17:14] "GET /static/css/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [17/Jan/2025 19:17:14] "GET /static/js/script.js HTTP/1.1" 304 -
127.0.0.1 - - [17/Jan/2025 19:17:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2025 19:17:15] "GET /static/images/image.jpg HTTP/1.1" 304 -
127.0.0.1 - - [17/Jan/2025 19:17:21] "GET / HTTP/1.1" 200 -
127.0.0.

1/1 [==============================] - 22s 22s/step


127.0.0.1 - - [17/Jan/2025 19:18:04] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2025 19:18:04] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2025 19:18:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2025 19:18:09] "GET /static/css/styles.css HTTP/1.1" 304 -
127.0.0.1 - - [17/Jan/2025 19:18:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jan/2025 19:18:09] "GET /static/images/image.jpg HTTP/1.1" 304 -
127.0.0.1 - - [17/Jan/2025 19:18:09] "GET /static/js/script.js HTTP/1.1" 304 -
127.0.0.1 - - [17/Jan/2025 19:18:13] "POST /predict HTTP/1.1" 200 -


1/1 [==============================] - 0s 47ms/step


127.0.0.1 - - [17/Jan/2025 19:18:23] "POST /predict HTTP/1.1" 200 -
